<span style="color:#888888">Copyright (c) 2014-2025 National Technology and Engineering Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525 with National Technology and Engineering Solutions of Sandia, LLC, the U.S. Government retains certain rights in this software.     Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="color:#888888">1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.</span>

<span style="color:#888888">2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.</span>

<span style="color:#888888">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# <span style="color:#0054a8">**Tutorial 2:**</span> <span style="color:#555555">How to Create Trajectories from a Delimited File</span>

In [ ]:
import tracktable.examples.tutorials.tutorial_helper as tutorial 

## Purpose

This notebook demonstrates how to create Tracktable trajectories from points stored in a delimited text (CSV, TSV, etc.) data file.  A data file must contain the following columns in order to be compatible with Tracktable:

* **<span style="color:#00add0">an identifier</span>** that is unique to each object
* **<span style="color:#00add0">a timestamp</span>**
* **<span style="color:#00add0">longitude</span>**
* **<span style="color:#00add0">latitude</span>**

Both ordering and headers for these columns can vary, but they must exist in the file.  Each row of the data file should represent the information for a single trajectory point.

**<span style="color:#81062e">IMPORTANT:</span>** delimited files must be **sorted in increasing order by timestamp** to be compatible with Tracktable.

*Note:* If you want to create individual `TrajectoryPoint` objects but not assemble them into `Trajectory` objects, please see [Tutorial 1](Tutorial_01_Points_From_File.ipynb).

## Step 1: Set up a TrajectoryPointReader object.

We will use the provided example data $^1$ for this tutorial.  We'll re-use the code from [Tutorial 1](Tutorial_01_Points_From_File.ipynb) to create a `TrajectoryPointReader` object.

In [ ]:
reader = tutorial.create_point_reader()

## Step 2: Create an AssembleTrajectoryFromPoints object.

This will build trajectories from the individual points.

In [ ]:
from tracktable.applications.assemble_trajectories import AssembleTrajectoryFromPoints

builder = AssembleTrajectoryFromPoints()

The builder expects an iterable of points as its input.  As we saw in [Tutorial 1](Tutorial_01_Points_From_File.ipynb), the reader provides that iterable.

In [ ]:
builder.input = reader

### *Identifying New Trajectories*

The trajectory assembler turns points into trajectories by grouping them according to their object ID.  It has three additional parameters.  *Separation time* and *separation distance* control whether two points with the same object ID belong to the same trajectory.  The *minimum point count* controls whether or not to discard too-short trajectories.



*Separation Distance*: How far apart (in km) should sequential points (with the same object ID) have to be before we consider them separate trajectories?  This parameter is optional and defaults to `None`, which disables the separation distance check.

In [ ]:
builder.separation_distance = 10 # km

*Separation Time*: How far apart (in time) should sequential points (with the same object ID) have to be before we consider them separate trajectories?  This parameter is optional and defaults to 30 minutes.  Setting it to `None` disables the separation time check.

In [ ]:
from datetime import timedelta

builder.separation_time = timedelta(minutes = 20)

*Minimum Point Count*: What is the minimum number of points that a trajectory must have?  Any trajectories assembled with fewer than this number will be discarded.  This parameter is optional and defaults to 2 points.  Setting it to 0 or `None` disables the minimum length check.

In [ ]:
builder.minimum_length = 5 # points

## Step 3: Assemble Trajectories from Point Data

Like the point reader, the trajectory builder provides its output as an iterable.  We pull the contents of that iterable into a list so that we can access it at our convenience.

In [ ]:
trajectories = list(builder)

How many trajectories do we have?

In [ ]:
len(trajectories)

## Step 4: Accessing Trajectory Information

For each `trajectory`, we can access the following information:

* `trajectory.object_id`: a string identifier that is unique to each moving object.  It is possible (and common, in some data sets) for multiple trajectories to share a single object ID.
* `trajectory.trajectory_id`: a string identifier that is mostly-unique to each trajectory, created by concatenating the object ID, start timestamp and end timestamp together

This is demonstrated below for our first ten trajectories.

In [ ]:
for trajectory in trajectories[:10]:
    object_id      = trajectory.object_id
    trajectory_id  = trajectory.trajectory_id
    
    print(f'Object ID: {object_id}')
    print(f'Trajectory ID: {trajectory_id}\n')

## Step 5: Accessing Trajectory Point Information

Let's look at just the first trajectory in our list:

In [ ]:
trajectory = trajectories[0]

Trajectory points can be accessed in a trajectory object using list indexing.  So, we can get the first point in our trajectory as follows:

In [ ]:
trajectory_point = trajectory[0]

The information from the required columns of the csv can be accessed for a single `trajectory_point` object as

* **<span style="color:#00add0">unique object identifier:</span>** `trajectory_point.object_id`
* **<span style="color:#00add0">timestamp:</span>** `trajectory_point.timestamp`
* **<span style="color:#00add0">longitude:</span>** `trajectory_point[0]`
* **<span style="color:#00add0">latitude:</span>** `trajectory_point[1]`

The optional column information is available through the member variable `properties` as follows: `trajectory_point.properties['what-you-named-it']`.

Below, we access all of the information stored in our `trajectory_point` object.

In [ ]:
object_id    = trajectory_point.object_id
timestamp    = trajectory_point.timestamp
longitude    = trajectory_point[0]
latitude     = trajectory_point[1]
heading      = trajectory_point.properties["heading"]
vessel_name  = trajectory_point.properties["vessel-name"]
eta          = trajectory_point.properties["eta"]

print(f'Unique ID: {object_id}')
print(f'Timestamp: {timestamp}')
print(f'Longitude: {longitude}')
print(f'Latitude: {latitude}')
print(f'Heading: {heading}')
print(f'Vessel Name: {vessel_name}')
print(f'ETA: {eta}\n')

## What can we do with trajectories in Tracktable?

* Compact trajectory storage using `.traj` files is discussed in Tutorials [3 (Write Trajectories to File)](Tutorial_03_Write_Trajectories_To_File.ipynb) & [4 (Read Traj File)](Tutorial_04_Read_Traj_File.ipynb).
* Tutorials [5A (Interactive Visualization)](Tutorial_05A_Interactive_Visualization.ipynb) & [5B (Static Visualization)](Tutorial_05B.ipynb) demonstrate how to visualize trajectories with Tracktable.
* Filtering trajectories using the capabilities provided by the `geomath` module is shown in [Tutorial 6 (Trajectory Filtering)](Tutorial_06.ipynb).

<span style="color:gray">$^1$ Bureau of Ocean Energy Management (BOEM) and National Oceanic and Atmospheric Administration (NOAA). MarineCadastre.gov. *AIS Data for 2020.* Retrieved February 2021 from [marinecadastre.gov/data](https://marinecadastre.gov/data/).  Trimmed down to the first hour of June 30, 2020, restricted to in NY Harbor.</span>